In [405]:
from random import randint
import torch
import torch.nn.functional as F
from torch import nn
import d2l.torch as d2l

seq = []

for i in range(1000 // 5):
    # 每次生成长度为 10 的连续序列
    start = randint(0, 10)  # [0, 10] 闭区间随机一个数值
    for j in range(start, start+5):
        seq.append(j)

T = len(seq)  # 序列数据 seq 的长度
num_steps = 20  # 时间步长
num_classes = 20  # 有多少个种类的词元;用于生成 ont-hot 编码

dtype = torch.int64

x = torch.tensor(seq, dtype=dtype)
X = torch.zeros((T-num_steps, num_steps), dtype=dtype)
Y = torch.zeros((T-num_steps, num_steps), dtype=dtype)

for i in range(num_steps):
    X[:, i] = x[i:T-num_steps+i]
    Y[:, i] = x[i+1:T-num_steps+i+1]

batch_size = 50  # 批量大小

sample_epochs = X.shape[0] // batch_size
num_samples = sample_epochs * batch_size

from torch.utils import data
dataset = data.TensorDataset(X[:num_samples, :], Y[:num_samples, :])
data_iter =data.DataLoader(dataset=dataset, batch_size=batch_size)

#### 官方模型

In [406]:
num_hiddens = 256
num_classes = 20
bacth_size = 50
num_steps = 20

# len(vocab) 是 RNN 的输入输出的大小 也是 序列的长度 时间步数
# num_hiddens 是 RNN 这一层的 单元个数
rnn_layer = nn.RNN(num_classes, num_hiddens)

# 我们使用张量来初始化隐状态，它的形状是（隐藏层数，批量大小，隐藏单元数）
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 50, 256])

In [407]:
X = torch.rand(size=(num_steps, batch_size, num_classes))
Y, state_new = rnn_layer(X, state)
# Y （时间步数，批量，隐藏层输出=单元数）
Y.shape, state_new.shape

(torch.Size([20, 50, 256]), torch.Size([1, 50, 256]))

In [408]:
#@save
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 它的输出形状是(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

#### 训练和预测

In [409]:
device = d2l.try_gpu()
net = RNNModel(rnn_layer, vocab_size=num_classes)
net = net.to(device)

num_epochs, lr = 500, 1
updater = torch.optim.SGD(net.parameters(), lr=0.05)
loss = nn.CrossEntropyLoss()

In [410]:
# net, train_iter, loss, updater, device, use_random_iter

# for epoch in range(100):
#     ppl, speed = d2l.train_epoch_ch8(net, data_iter, loss, updater, device, False)
#     print(ppl, ' and ', speed)

In [411]:
state = net.begin_state(batch_size=batch_size, device=device)

for i in range(2):
    for X, Y in data_iter:
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        # 这个 state 为啥会有梯度，我也是没法理解啊
        print(state.requires_grad)
        state.detach_()
        
        # 通过这个方式也可以
        # 所以说果然是 state 造成的问题
        # state = net.begin_state(batch_size=batch_size, device=device)


        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        # print(l)
        updater.zero_grad()
        l.backward()
        updater.step()

False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [412]:
def predict_ch8(prefix, num_preds, net, device):  #@save
    """
    在prefix后面生成新字符
    num_preds: 后续要生成多少个 词（本节的次元是 字符）
    """
    # batch_size = 1 ==> 对一个字符串做预测
    state = net.begin_state(batch_size=1, device=device)
    outputs = [int(prefix[0])]  # 把 prefix[0] 对应的 数值映射 放进 outputs 中
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    
    for y in prefix[1:]:  # 预热期 ==> 把 prefix 中的信息初始化到 state 中
        # y 是从 prefix 的 第1号 元素开始取值的
        # 当 y 取 prefix[1] 的时候，此时 get_input() 得到的输入是 prefix[0] 
        # 刚好是错开的，是没有问题的
        # 之所以 state 也是网络的输入参数
        # 就是使用 state 保存了 prefix 的信息，而 W_hh 是帮助生成 state 用的
        inputs = get_input()
        _, state = net(inputs, state)
        outputs.append(int(y))

    # 从这儿开始是预测 prefix 后边的字符
    # 每次都用上一次输出的值作为下一次输入的值
    # 上一次的输出值是一个时间步上的值，而不是一个序列
    # 这刚好和在训练模型时的思路是一致的，使用了时间序列中的
    # 某一个时间步上的值的 one-hot 编码作为输入
    # 因此这里的 RNN 在真正预测的核心部分，它的输入依旧是时间序列中的
    # 某个时间步的值，但用 ont-hot 值来表达
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))

    # 把 index 的值 转化成其 对应的 词（这里是字符）
    return outputs

In [413]:
predict_ch8('123', 3, net, device)

[1, 2, 3, 4, 5, 6]

In [414]:
predict_ch8('123', 10, net, device)

[1, 2, 3, 4, 5, 6, 5, 6, 6, 5, 6, 6, 5]

In [415]:
x1 = torch.tensor([1], dtype=torch.float32, requires_grad=True)
x2 = torch.tensor([1], dtype=torch.float32, requires_grad=True)

state = torch.tensor([4], dtype=torch.float32)

y1 = 2 * x1 + state * x2
# 如果在 y1 执行 backward 的时候不保存计算图 retain_graph=True
# 那么在执行 y2.backward() 的时候，由于 y1 进行过反向传播了
# 所以计算图就已经被自动销毁，丢失了
# 因此就不能完成反向传播的任务了
# 只有当再一次执行了前向传播，才会再次建立计算图，这时可以反向传播
y1.backward(retain_graph=True)
x1.grad.zero_(), x2.grad.zero_()
y2 = 2 * x1 + y1 * x2
y2.backward()

"""
对于 RNN 中的 state 也是一样的
在同一批量中即使每次循环生成每个时间步的 state 时，每次都会和 权重参数建立联系
但是在同一个批量中进行这个过程是在完成前向传播且建立计算图的过程
这个过程中并没有进行反向传播，所以计算图就一直存在

而当完成一个批量之后，上一个批量的最终 state 在下一个批量中参与运算
而上一个批量的 state 所关联的计算图已经被销毁了
所以下一个批量若进行反向传播的话，就会报错了
所以要使用 detach_ 来完成与上一个批量计算中的计算图分离的操作才可以

而在 RNN 中使用 backward(retain_graph=True) 会产生新的问题
在每一个带有梯度变量的 tensor 中记录了 version
若 version 不为 1 ，也是不能完成梯度计算的??
"""

'\n对于 RNN 中的 state 也是一样的\n在同一批量中即使每次循环生成每个时间步的 state 时，每次都会和 权重参数建立联系\n但是在同一个批量中进行这个过程是在完成前向传播且建立计算图的过程\n这个过程中并没有进行反向传播，所以计算图就一直存在\n\n而当完成一个批量之后，上一个批量的最终 state 在下一个批量中参与运算\n而上一个批量的 state 所关联的计算图已经被销毁了\n所以下一个批量若进行反向传播的话，就会报错了\n所以要使用 detach_ 来完成与上一个批量计算中的计算图分离的操作才可以\n\n而在 RNN 中使用 backward(retain_graph=True) 会产生新的问题\n在每一个带有梯度变量的 tensor 中记录了 version\n若 version 不为 1 ，也是不能完成梯度计算的??\n'

In [416]:
x1.grad, x2.grad

(tensor([4.]), tensor([10.]))

In [417]:
x1 = torch.tensor([1], dtype=torch.float32, requires_grad=True)
x2 = torch.tensor([1], dtype=torch.float32, requires_grad=True)
state = torch.tensor([4], dtype=torch.float32)
x1._version, x2._version, x1.requires_grad

(0, 0, True)

In [418]:
y1 = 2 * x1 + state * x2
# 如果在 y1 执行 backward 的时候不保存计算图 retain_graph=True
# 那么在执行 y2.backward() 的时候，由于 y1 进行过反向传播了
# 所以计算图就已经被自动销毁，丢失了
# 因此就不能完成反向传播的任务了
# 只有当再一次执行了前向传播，才会再次建立计算图，这时可以反向传播
y1.backward(retain_graph=True)
x1.grad.zero_(), x2.grad.zero_()
y2 = 2 * x1 + y1 * x2
y2.backward()

In [419]:
x1._version, x2._version

(0, 0)

In [420]:
x1.detach_()
x2.detach_()

x1 += 1
x2 += 1

In [421]:
x1._version, x2._version

(1, 1)

In [422]:
x1.requires_grad_()

tensor([2.], requires_grad=True)

In [423]:
x2.requires_grad_()

tensor([2.], requires_grad=True)

In [424]:
x1.detach_()
x2.detach_()

x1 += 1
x2 += 1

In [425]:
x1._version, x2._version

(2, 2)

In [426]:
x1 = torch.tensor([1], dtype=torch.float32, requires_grad=True)
x2 = torch.tensor([1], dtype=torch.float32, requires_grad=True)
state = torch.tensor([4], dtype=torch.float32)
x1._version, x2._version, x1.requires_grad

(0, 0, True)

In [427]:
y1 = 2 * x1 + state * x2
y2 = 2 * x1 + y1 * x2

In [429]:
y1.requires_grad, y2.requires_grad

(True, True)

In [432]:
y2.backward()

In [434]:
y1.requires_grad  # 为什么 y1 会附有梯度？

True